Q1. Given a positive integer “k” and a list of integer numbers, look for the numbers within the list, that are less than k. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case.

In [86]:
from qiskit import QuantumRegister, ClassicalRegister, transpile, QuantumCircuit
from qiskit.providers import basic_provider

#Defining a bit comparator circuit
def bit_compare():
    qr = QuantumRegister(2, "bits")
    aux = QuantumRegister(2, "aux")
    
    qc = QuantumCircuit(qr, aux)
    qc.x(qr[1])
    qc.mcx(qr, aux[0])
    qc.x(qr[0])
    qc.x(qr[1])
    qc.mcx(qr, aux[1])
    qc.x(qr[0])
    
    return qc

#Defining the circuit for bit comparison
def less_than_k(k,lst):
    
    m = len(bin(max(k,max(lst)))[2:]) #Getting the largest number in list to get number of qubits needed to draw the circuit
    
    flist=[] #To store numbers less than k
    
    for a in range(len(lst)):
        num1_b=bin(k)[2:].zfill(m)
        num2_b=bin(lst[a])[2:].zfill(m)
        q=QuantumRegister(m*2,'q')
        c=ClassicalRegister(m+2,'c')
        qc=QuantumCircuit(q,c)
        for i in range(m):
            if num1_b[i] == '1':
                qc.x(2*i)
            if num2_b[i] == '1':
                qc.x(2*i+1)
            qc.cx(2*i,2*i+1)
            qc.measure(q[2*i+1],c[i])
            qc.barrier()
        backend = basic_provider.BasicProvider().get_backend('basic_simulator')
        # Do the simulation, returning the result
        res = transpile(qc,backend)
        job=backend.run(res)
        results=job.result()
        #get the probability distribution
        counts = results.get_counts()
        lst1=str(list(counts.keys())[0])[::-1]

        q1=QuantumRegister(4,'q1')
        c1=ClassicalRegister(2,'c1')
        qcf=QuantumCircuit(q1,c1)
        for i,j in enumerate(lst1):
            if j == '1':
                if num1_b[i] == '1':
                    qcf.x(0)
                if num2_b[i] == '1':
                    qcf.x(1)
                qcf.append(bit_compare(),[q1[0],q1[1],q1[2],q1[3]])
                break
        qcf.measure([q1[2],q1[3]],[c1[0],c1[1]])
    # Do the simulation, returning the result
        res1 = transpile(qcf,backend)
        job1=backend.run(res1)
        results1=job1.result()
    # get the probability distribution
        counts1 = results1.get_counts()
        final=str(list(counts1.keys())[0])
        if final == '01':
            flist.append(lst[a])
    return flist

print(less_than_k(6,[8,9,7,4,1,5]))

[4, 1, 5]


Source: Oliveira, David & Ramos, Rubens. (2007). Quantum bit string comparator: Circuits and applications. Quantum Computers and Computing. 7.

In [88]:
#Quantum Circuit for the bit comparator is shown below
print(bit_compare())

                  ┌───┐     ┌───┐
bits_0: ───────■──┤ X ├──■──┤ X ├
        ┌───┐  │  ├───┤  │  └───┘
bits_1: ┤ X ├──■──┤ X ├──■───────
        └───┘┌─┴─┐└───┘  │       
 aux_0: ─────┤ X ├───────┼───────
             └───┘     ┌─┴─┐     
 aux_1: ───────────────┤ X ├─────
                       └───┘     


In [94]:
#Circuit for comparison of each number is drawn as such
def circuit(num1,num2):
    m = len(bin(max(num1,num2))[2:])
    num1_b=bin(num1)[2:].zfill(m)
    num2_b=bin(num2)[2:].zfill(m)
    q=QuantumRegister(m*2,'q')
    c=ClassicalRegister(m+2,'c')
    qc=QuantumCircuit(q,c)
    for i in range(m):
        if num1_b[i] == '1':
            qc.x(2*i)
        if num2_b[i] == '1':
            qc.x(2*i+1)
        qc.cx(2*i,2*i+1)
        qc.measure(q[2*i+1],c[i])
        qc.barrier()
    return qc

print(circuit(2,3))

     ┌───┐         ░               ░ 
q_0: ┤ X ├──■──────░───────────────░─
     ├───┤┌─┴─┐┌─┐ ░               ░ 
q_1: ┤ X ├┤ X ├┤M├─░───────────────░─
     └───┘└───┘└╥┘ ░               ░ 
q_2: ───────────╫──░────────■──────░─
                ║  ░ ┌───┐┌─┴─┐┌─┐ ░ 
q_3: ───────────╫──░─┤ X ├┤ X ├┤M├─░─
                ║  ░ └───┘└───┘└╥┘ ░ 
c: 4/═══════════╩═══════════════╩════
                0               1    


The appropriate number of qubits required are number of bits needed to represent the bigegst number out of all given numbers plus 4 since we need 4 more qubits to apply the bit comparator circuit
This algorithm might seem like it is worse than its classical counterpart but it should be noted that this circuit uses less resources since we make use of CNOT gate as an XOR gate and deliver the result using only the bit where the XOR gate returns the value 1 meaning the two bits at the same position have different values so we only need to compare them. Making an XOR gate in classical computer uses number of NAND and NOR gates and could be resource intensive while in this quantum algorithm, it simply uses the CNOT gate as an XOR gate.